In [99]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc
import h5py
import os

In [100]:
def get_vid(fnames):
    shape = (720, 1280)
    src_path = '/data/vision/phillipi/gen-models/svg/dataset/omnipush/train'
    tar_path = '/data/vision/phillipi/gen-models/TAKE_THIS_straight_push_all_shapes_no_weight/abs'
    fnames = [fname.replace(src_path, tar_path) for fname in fnames]
    fnames = [fname.replace('/motion_surface', '/normal/motion_surface') for fname in fnames]
    fnames = [fname + '_sync.h5' for fname in fnames]

    vids = []
    for fname in fnames:
        f = h5py.File(fname, 'r')
        vids.append(f.get('RGB_images').value[:-5])        
        f.close()
    
    for i, vid in enumerate(vids):
        if i == 0:
            vid_stitched = np.copy(vid)
        else:
            vid_stitched = np.concatenate([vid_stitched, vid], axis=0)
    return vid_stitched

def save_seqs(vid_stitched, shape_name):
    width = 1280
    height = 720
    FPS = 12
    seconds = vid_stitched.shape[0] / 12
    
    fourcc = VideoWriter_fourcc(*'MP42')
    video = VideoWriter('./{}.mp4'.format(shape_name), fourcc, float(FPS), (width, height))
    for i in range(vid_stitched.shape[0]):
        video.write(vid_stitched[i])

In [92]:
DATASET_PATH = '/data/vision/phillipi/gen-models/TAKE_THIS_straight_push_all_shapes_no_weight/abs'
shape_names = os.listdir(DATASET_PATH)
seq_shape = np.load('seq_shape.npy').item()

In [101]:
for shape_name in shape_names:
    if shape_name in ['done_shapes', 'proccessed_data', 'wrong_data_vicon']:
        continue
    seqs = seq_shape[shape_name]
    for i in range(5):
        seq = seqs[i]
        vid_stitched = get_vid(seq)
        save_seqs(vid_stitched, shape_name + '_{}'.format(i))

KeyboardInterrupt: 